In [36]:
import os
import sys

import numpy as np
import napari
import h5py
import pandas as pd
import skimage.io
from pathlib import Path

In [37]:
# Import path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path = str(Path.cwd().parents[0] / "src")
if module_path not in sys.path:
    sys.path.append(module_path)
    
from config import *

In [38]:
from skimage import exposure

def get_masks(mask_folder):
    '''
    Function to get all mask from mask forlder
    '''
    # Read masks
    masks = {}

    for (dirpath, dirnames, filenames) in os.walk(mask_folder):
        for name in sorted(filenames):
            if "tif" in name:
                filename = os.path.join(dirpath, name)
                img = skimage.io.imread(filename)
                condition = name.split('_')[1][:2]
                masks[condition] = img
            else:
                continue
    return masks

def get_imgs(file_path, name):
    f = h5py.File(file_path, 'r')
    imgs = f[name]
    labels = list(f[name].attrs['labels'])
    return imgs, labels

def blended_img(viewer):
    import napari
    import numpy as np
    
    blended = np.zeros(viewer.layers[0].data.shape + (4,))
    for layer in viewer.layers:
        # normalize data by clims
        normalized_data = (layer.data - layer.contrast_limits[0]) / (
        layer.contrast_limits[1] - layer.contrast_limits[0]
    )
        colormapped_data = layer.colormap.map(normalized_data.flatten())
        colormapped_data = colormapped_data.reshape(normalized_data.shape + (4,))

        blended = blended + colormapped_data
    
    blended = blended[:,:,:3]
    p2, p98 = np.percentile(blended, (0.1, 99.9))
    img_rescale = exposure.rescale_intensity(blended, in_range=(p2, p98), out_range=(0,255)).astype(np.uint8)
    return img_rescale

In [39]:
data_ROI = data_dir / 'ROI_new'
h5_data = data_ROI / f'TMA.hdf5'
df = pd.read_csv(data_ROI / f"info.csv")

ROIs = df.ROI.unique()
masks = get_masks(data_ROI / 'masks')

In [40]:
# print(markers)

In [41]:
for roi in ROIs[-1:]:

    imgs, markers = get_imgs(h5_data, str(roi))
    mask = masks[str(roi)]
    
    ## 1
    marker_subset = ['SMA', 'Vimentin', 'Pankeratin', 'E-cadherin', 'COL1']
    colors = ['yellow', 'bop orange', 'magenta', 'red', 'cyan']
    marker_index = [markers.index(marker) for marker in marker_subset] 

    viewer = napari.view_image(imgs[marker_index, :,:], channel_axis=0, name=marker_subset, colormap=colors, show=False)

    img_RGB = blended_img(viewer)
    skimage.io.imsave(f'{roi}_1.png', img_RGB)

    ## 2
    marker_subset = ['CD163', 'CD206', 'CD68', 'HLADR']
    colors = ['red', 'magenta', 'green','cyan']
    marker_index = [markers.index(marker) for marker in marker_subset] 

    viewer = napari.view_image(imgs[marker_index, :,:], channel_axis=0, name=marker_subset, colormap=colors, show=False)
    img_RGB = blended_img(viewer)
    skimage.io.imsave(f'{roi}_2.png', img_RGB)

    ## 3
    marker_subset = ['TCF1', 'CD103', 'CD8']
    colors = ['red', 'magenta', 'cyan']
    marker_index = [markers.index(marker) for marker in marker_subset] 

    viewer = napari.view_image(imgs[marker_index, :,:], channel_axis=0, name=marker_subset, colormap=colors, show=False)
    img_RGB = blended_img(viewer)
    skimage.io.imsave(f'{roi}_3.png', img_RGB)

    ## 4
    marker_subset = ['TCF1', 'CD103', 'CD4', 'CD3']
    colors = ['red', 'magenta', 'cyan', 'yellow']
    marker_index = [markers.index(marker) for marker in marker_subset] 

    viewer = napari.view_image(imgs[marker_index, :,:], channel_axis=0, name=marker_subset, colormap=colors, show=False)
    img_RGB = blended_img(viewer)
    skimage.io.imsave(f'{roi}_4.png', img_RGB)
    


In [42]:
viewer = napari.view_image(imgs, channel_axis=0, name=markers)